In [1]:
import requests, json, math, sqlite3
API_TOKEN = ""
conn = sqlite3.connect('C:/sqlite/UnitedStations.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS stations
             (ID text, state text, name text, elev real, mindate text, maxdate text, lat real, long real)''')

In [2]:
req_states = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=ST&limit=52", 
                          headers={"token":API_TOKEN})
states_raw = json.loads(req_states.text)
states = states_raw['results']
state_stations = {}
for s in states:
    state_stations[s['id']] = s['name']
    

In [ ]:
for key in state_stations:
    i = 0
    state = state_stations[key]
    
    while True:
        req = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?locationid={}&limit=1000&offset={}"
            .format(key,i * 1000 + 1), headers={"token":API_TOKEN})
        req_text = json.loads(req.text)
        number_stations = req_text['metadata']['resultset']['count']
        stations = req_text['results']
        
        for s in stations:
            if 'elevation' in s:
                t = (s['id'], state, s['name'], s['elevation'], s['mindate'], s['maxdate'], s['latitude'], s['longitude'])
            else:
                t = (s['id'], state, s['name'], None, s['mindate'], s['maxdate'], s['latitude'], s['longitude'])
            c.execute("INSERT INTO stations VALUES (?,?,?,?,?,?,?,?)", t)
      
        i = i + 1
        if(i >= math.ceil(number_stations/1000)):
            break
            
conn.commit()


In [8]:
# #Total number of stations
# total_stations = 0

# for key in state_stations:
#     hmm = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?locationid={}".format(key)
#             , headers={"token":"KxpzfqKFVIjaYiggsqRLQWXAsuAxthBO"})
#     hmm_text = json.loads(hmm.text)
#     total_stations += hmm_text['metadata']['resultset']['count']

# print("There are a total of {} stations in the United States. :)".format(total_stations))
    

There are a total of 83675 stations in the United States. :)
